In [7]:
#import math
import copy
#import pyparsing
import ezdxf
from ezdxf.enums import TextEntityAlignment
import datetime
import os
from exif import Image as Exif_Image
import sys
from PIL import Image

root_directory = ("C:\\Users\\Carlos de Gouveia\\My Drive\\Projects\\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\\Snagging")
global_line_density = (.1,50)
default_image = ""
default_text_title = ""
default_text_comment = ""
default_text_height = 2
image_filename_extensions = ('jpg','jpeg')
keyword_brackets = ('[', '(')

def find_snagged_image_files(root_path):
    file_list=[]
    if os.path.exists(root_path):
        #print("Root path is found: " + root_path)
        #print('Busy looking for images')
        for (root,dirs,files) in os.walk(root_path, topdown=1): 
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg')):
                    if len(file.split(".")) > 2:
                        file_found_with_path=os.path.join(root, file)
                        file_list.append(file_found_with_path)
        #print('Done')
    else:
        print("Root path is not found. Please check path accessibility: " + root_path)
    return(file_list)

def test_for_keyword_brackets(filename_part):
    if "[" in filename_part:
        if "]" in filename_part:
            result = True
        else:
            result = False
    else: 
        result = False
    return(result)

def get_snag_comment_from_title(image_file_with_path):
    filename_parts = image_file_with_path.split(".")
    #print(filename_parts)
    snag_comment = ""
    for part in filename_parts:
        if part.lower() in image_filename_extensions:
            part = ""
        if test_for_keyword_brackets(part):
            part = ""
        if "\\" in part:
            part = ""
        if "IMG_" in part:
            part = ""
        if part != "":
            snag_comment = snag_comment + part + "."
    return(snag_comment)

def get_snag_keyword_from_title(image_file_with_path):
    filename_parts = image_file_with_path.split("[")
    snag_keywords = [""]
    for phrase in filename_parts:
        if "]" in phrase:
            snag_keywords.append(phrase.split(']')[0])
    #print("snag keyowrds", snag_keywords)
    return(snag_keywords)

def get_location_tree_from_path(root_directory, image_file_with_path):
    root_parts = root_directory.split("\\")
    filename_parts = image_file_with_path.split("\\")
    test = True
    location_tree = []
    count_from_left = len(root_parts)
    while test == True and count_from_left != len(filename_parts):
        if "." in filename_parts[count_from_left]:
            test = False
        else:
            location_tree.append(filename_parts[count_from_left])
        count_from_left += 1
    return(location_tree)
    
#def get_unique_id_from_title

def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
    if ref == 'S' or ref == 'W':
        decimal_degrees = -decimal_degrees
    return decimal_degrees

def get_image_exif(image_file_with_path):
    exif_list=["Date","coords_lat","coords_long",["null","null"]]
    #print (image_file_with_path)
    with open(image_file_with_path, 'rb') as src:
        img = Exif_Image(src)
        date=('')
        if img.has_exif:
            try:
                exif_list[0] = img.datetime_original
            except:
                pass
            try:
                exif_list[1] = round(decimal_coords(img.gps_latitude,img.gps_latitude_ref),4)
                exif_list[2] = round(decimal_coords(img.gps_longitude,img.gps_longitude_ref),4)
            except:
                pass
            try:
                exif_list[3][0] = img.image_description
            except:
                pass
            try:
                exif_list[3][1] = img.xp_comment
            except:
                pass
    return(exif_list)

def get_work_scope(work_scope_template, keyword_texts):
    work_scope = []
    for keyword in keyword_texts:
        for key in work_scope_template:
            if keyword in key[0]:
                work_scope_text="BLANK"
                work_scope_colour=key[2]
                work_scope.append([work_scope_text, work_scope_colour])
    #print("Work Scope fffffffffffffffffffff",work_scope)
    return(work_scope)

def create_work_scope_score(work_scope_template, sticker_template):
    count_col = 0
    count_row = 0
    sticker_row = [0] * (len(sticker_template) + 1)
    work_scope_score = []
    work_scope_score.append(copy.deepcopy(sticker_row))
    for row in work_scope_template:
        work_scope_score.append(copy.deepcopy(sticker_row))
    return(work_scope_score)

def keep_score_work_scope(work_scope_template, sticker_template, keyword_texts, work_scope_score):
    for keyword in keyword_texts:
        count_col = 0
        count_row = 0
        for key_sticker in sticker_template:
            count_col += 1
            if keyword in key_sticker[0]:
                work_scope_score[count_row][count_col] += 1
        count_col = 0
        count_row = 0
        for key_scope in work_scope_template:
            count_row += 1
            if keyword in key_scope[0]:
                work_scope_score[count_row][count_col] += 1
    for keyword in keyword_texts:
        count_col = 0
        count_row = 0
        for key_sticker in sticker_template:
            count_col += 1
            if keyword in key_sticker[0]:
                for keyword_sub in keyword_texts:
                    count_row = 0
                    for key_scope in work_scope_template:
                        count_row += 1
                        if keyword_sub in key_scope[0]:
                            work_scope_score[count_row][count_col] += 1
    return(work_scope_score)

def print_work_scope(centre_pos_x, centre_pos_y, keyword_text, colour):
    
    hatch = a_block.add_hatch(color=colour)
    path = ((centre_pos_x-2, centre_pos_y-2), 
            (centre_pos_x-2, centre_pos_y+2), 
            (centre_pos_x+6, centre_pos_y+2), 
            (centre_pos_x+6, centre_pos_y-2)
           )
    hatch.paths.add_polyline_path(path, is_closed=True)    
    if keyword_text != "BLANK":
        mtext = a_block.add_mtext("", dxfattribs={'style': 'ARIALBD'})
        mtext += keyword_text
        mtext.dxf.char_height = 1             
        mtext.dxf.width = 48 #1mm gap from box boundary
        mtext.set_location(insert=(centre_pos_x+7, centre_pos_y), rotation=0, attachment_point=4) #1mm gap from box boundary

def print_score(centre_pos_x, centre_pos_y, keyword_text, size, colour):
    mtext = a_block.add_mtext("", dxfattribs={'style': 'ARIALBD'})
    mtext += keyword_text
    mtext.dxf.char_height = size
    mtext.dxf.color = colour      
    mtext.dxf.width = size
    mtext.set_location(insert=(centre_pos_x, centre_pos_y), rotation=0, attachment_point=5)
        
def get_badge(badge_template, keyword_texts):
    badge = []
    for keyword in keyword_texts:
        for key in badge_template:
            if keyword in key[0]:
                badge_text=key[1]
                badge_colour=key[2]
                badge.append([badge_text, badge_colour])
                #print("Badge ", badge_text, badge_colour)
    return(badge)

def print_badge(centre_pos_x, centre_pos_y, keyword_text, colour):
    mtext = a_block.add_mtext("", dxfattribs={'style': 'ARIALBD'})
    mtext += keyword_text
    mtext.dxf.char_height = 12             
    mtext.set_bg_color(colour, scale=1)
    mtext.dxf.width = 100 #1mm gap from box boundary
    mtext.set_location(insert=(centre_pos_x, centre_pos_y), rotation=0, attachment_point=5) #1mm gap from box boundary

def get_sticker(sticker_template, keyword_texts): 
    sticker = []
    for keyword in keyword_texts:
        for key in sticker_template:
            if keyword in key[0]:
                sticker_text=key[1]
                sticker_colour=key[2]
                #print("Stcker Colour",sticker_colour)
                sticker.append([sticker_text, sticker_colour])
                #print("Sticker ",sticker_text, sticker_colour)
    return(sticker)

def print_sticker(centre_pos_x, centre_pos_y, keyword_text, colour):
    #print("Keyword text ", keyword_text)
    
    hatch = a_block.add_hatch(color=colour)
    path = ((centre_pos_x-20, centre_pos_y-3.5), 
            (centre_pos_x-20, centre_pos_y+3.5), 
            (centre_pos_x+20, centre_pos_y+3.5), 
            (centre_pos_x+20, centre_pos_y-3.5)
           )
    hatch.paths.add_polyline_path(path, is_closed=True)

    mtext = a_block.add_mtext("", dxfattribs={'style': 'ARIALBD'})
    mtext += keyword_text
    mtext.dxf.char_height = 6             
    #mtext.set_bg_color(colour, scale=1)
    mtext.dxf.width = 100 #1mm gap from box boundary
    mtext.set_location(insert=(centre_pos_x, centre_pos_y), rotation=0, attachment_point=5) #1mm gap from box boundary

def start_new_drawing(drawing_file_with_path):
    # create a new DXF R2010 document
    snag_report_drawing = ezdxf.new("R2010", setup=True)
    snag_report_drawing.set_raster_variables(frame = 0, quality = 1, units = "m")
    # save the DXF document
    snag_report_drawing.saveas(drawing_file_with_path)

def create_drawing_template(drawing_file_with_path, page_number, template_entities):
    # Load existing drawing
    try:
        snag_report_drawing = ezdxf.readfile(drawing_file_with_path)
    except IOError:
        #print(f"Not a DXF file or a generic I/O error.")
        sys.exit(1)
    except ezdxf.DXFStructureError:
        #print(f"Invalid or corrupted DXF file.")
        sys.exit(2)
    # Create a block with the template name
    block_name = str(page_number)
    global a_block
    a_block = snag_report_drawing.blocks.new(name = block_name)
    # Add DXF entities to the block_name.
    
    # Very very first add hatches (so they go behind images in the title page)
    if template_entities["HATCH"] != [""]:
        for path in template_entities["HATCH"]:
            hatch = a_block.add_hatch(color=9)
            hatch.paths.add_polyline_path(path, is_closed=True)

    # First add Images and size to fit image block (and sent to back)
    snag_images = template_entities["SNAG_IMAGE"]
    for image in snag_images:
        points = image[0]
        #Find EXTENTS
        reset = 1
        for point in points:
            if reset == 1:
                x_min = point[0]
                x_max = point[0]
                y_min = point[1]
                y_max = point[1]
                reset = 0
            if x_min > point[0]:
                x_min = point[0]
            if x_max < point[0]:
                x_max = point[0]
            if y_min > point[1]:
                y_min = point[1]        
            if y_max < point[1]:
                y_max = point[1]
        image_file_path = image[1]
        if image_file_path == "default":
            image_file_path = default_image
        try:
            with Image.open(image_file_path) as img:
                my_image_def = snag_report_drawing.add_image_def(filename=image_file_path, size_in_pixel=img.size)      
                my_image = a_block.add_image(insert=(points[0][0],points[0][1]), size_in_units=((x_max-x_min), (y_max-y_min)), image_def=my_image_def, rotation=0)
                #print(points)
                #print("TESTER SHOULD BE CONSTANT",x_min,y_min)
                verts=[]
                scale = ((img.size[0]/(x_max-x_min)),(img.size[1]/(y_max-y_min)))
                count = 0
                for point in points:
                    if count > 1: #ignore first and second vertex, which is the insertion point and the opposite corner
                        verts.append(((point[0]-x_min)*scale[0],(point[1]-y_min)*scale[1]))
                    count +=1
                my_image.set_boundary_path(verts)
                my_image.dxf.clipping = 1
                #print("TEST",scale)
        except:
            #print("ERROR")
            pass
    # Second add Text so that it fits text box
    snag_texts = template_entities["SNAG_TEXT"]
    for snag in snag_texts:
        #print(snag)
        #Find text box extents
        if snag[0][0] < snag[1][0]:
            x_min = snag[0][0]
            x_max = snag[1][0]
        else:
            x_min = snag[1][0]
            x_max = snag[0][0]
        if snag[0][1] < snag[1][1]:
            y_min = snag[0][1]
            y_max = snag[1][1]
        else:
            y_min = snag[1][1]
            y_max = snag[0][1]
        title_text = snag[2]
        #print(title_text)
        comment_text = snag[3][3][0] + snag[3][3][1]
        date_text = snag[3][0]
        gps_lat_text = snag[3][1]
        gps_long_text = snag[3][2]

        if title_text == "default":
            title_text = default_text_title
        if comment_text == "default":
            comment_text = default_text_comment
        if comment_text == "defaultdefault":
            comment_text = default_text_comment
        if comment_text == "null":
            comment_text = default_text_comment
        if comment_text == "nullnull":
            comment_text = default_text_comment

        #Print comment    
        #print(comment_text)
        keyword_text = snag[4]
        mtext = a_block.add_mtext("", dxfattribs={'style': 'ARIALBD'})
        mtext += title_text
        mtext += "\P"
        mtext += comment_text
        if keyword_text != "TITLE":
            mtext.dxf.char_height = template_entities["TXT_PARAM"][0]             ###########default_text_height
        else:
            mtext.dxf.char_height = template_entities["TXT_PARAM"][2]
        mtext.dxf.width = x_max-x_min-1*2 #1mm gap from box boundary
        if keyword_text != "TITLE":
            mtext.set_location(insert=(x_min+1, y_max-1), rotation=0, attachment_point=template_entities["TXT_PARAM"][1]) #1mm gap from box boundary
        else:
            mtext.set_location(insert=(x_min+1, y_max-1), rotation=0, attachment_point=template_entities["TXT_PARAM"][3]) #1mm gap from box boundary
        #print date
        if date_text != "default":
            mtext = a_block.add_mtext("", dxfattribs={'style': 'ARIALBD'})
            mtext += "PHOTO TIMESTAMP: "
            mtext += date_text
            mtext.dxf.char_height = 1             ###########default_text_height
            mtext.dxf.width = (x_max-x_min)/2-1*2 #1mm gap from box boundary
            mtext.set_location(insert=(x_min+29.25+1, y_min+4.2-0.5), rotation=0, attachment_point=template_entities["TXT_PARAM"][1]) #1mm gap from box boundary
        
        #print GPS
        if gps_lat_text != "default" and gps_lat_text != "default":
            mtext = a_block.add_mtext("", dxfattribs={'style': 'ARIALBD'})
            mtext += "GPS LAT: "
            mtext +=  str(gps_lat_text)
            mtext += "  /  "
            mtext += "GPS LONG: "
            mtext +=  str(gps_long_text)
            mtext.dxf.char_height = 1             ###########default_text_height
            mtext.dxf.width = (x_max-x_min)/2-1*2 #1mm gap from box boundary
            mtext.set_location(insert=(x_min+29.25+1, y_min+4.2-0.5-2), rotation=0, attachment_point=template_entities["TXT_PARAM"][1]) #1mm gap from box boundary

    #print Work Scope 
    for image_work_scope in template_entities["WORK_SCOPE"]:
        #print("image_work_scope",image_work_scope)
        count = 0
        for work_scope in image_work_scope[2]:
            #print("Work_scope", work_scope)
            if work_scope[0] != "":
                #print("count", count)
                centre_pos_x = image_work_scope[0][0] + count * image_work_scope[1][0]
                centre_pos_y = image_work_scope[0][1] + count * image_work_scope[1][1]  
                #print(centre_pos_x)
                text = work_scope[0]
                colour = work_scope[1]
                #print("text", text)
                print_work_scope(centre_pos_x, centre_pos_y, text, colour)
                count += 1     

    #print SCORES 
    count = 0
    for score in template_entities["SCORE_TEXT"]:
        centre_pos_x = score[0][0]
        centre_pos_y = score[0][1]  
        text = score[1][0]
        size = score[1][1]
        colour = score[1][2]
        print_score(centre_pos_x, centre_pos_y, text, size, colour)
        count += 1                    

    #print Badge 
    for image_badge in template_entities["BADGE"]:
        count = 0
        for badge in image_badge[2]:
            if badge[0] != "":
                centre_ = image_badge[0][0] + count * image_badge[1][0]
                centre_pos_y = image_badge[0][1] + count * image_badge[1][1]  
                text = badge[0]
                colour = badge[1]
                print_badge(centre_pos_x, centre_pos_y, text, colour)
                count += 1
                    
    #print Sticker 
    for image_sticker in template_entities["STICKER"]:
        #print("image_sticker0000000000000000000000000000000000000000000000000000000", image_sticker[2])
        count = 0
        for sticker in image_sticker[2]:
            if sticker[0] != "":
                centre_pos_x = image_sticker[0][0] + count * image_sticker[1][0]
                centre_pos_y = image_sticker[0][1] + count * image_sticker[1][1]  
                text = sticker[0]
                colour = sticker[1]
                print_sticker(centre_pos_x, centre_pos_y, text, colour)
                #print("count", count)
                count += 1

    # Third add lines (and set to front)
    lines = template_entities["LINE"]
    for line in lines:
        start_point = line[0]
        end_point = line[1]
        width = line[2]
        how_wide=width[0]
        how_dense=width[1]
        delta_x = ((start_point[0]-end_point[0])/((start_point[1]-end_point[1])**2 + (start_point[0]-end_point[0])**2)**0.5)
        delta_y = ((start_point[1]-end_point[1])/((start_point[1]-end_point[1])**2 + (start_point[0]-end_point[0])**2)**0.5)
        count = 0
        while count <= (how_wide * how_dense)/2:
            start = ((start_point[0] - delta_y * count * (1/how_dense)),(start_point[1] + delta_x * count * (1/how_dense)))
            end = ((end_point[0] - delta_y * count * (1/how_dense)),(end_point[1] + delta_x * count * (1/how_dense)))
            a_block.add_line(start, end)
            count +=1
        count = 0
        while count <= (how_wide * how_dense)/2:
            start = ((start_point[0] + delta_y * count * (1/how_dense)),(start_point[1] - delta_x * count * (1/how_dense)))
            end = ((end_point[0] + delta_y * count * (1/how_dense)),(end_point[1] - delta_x * count * (1/how_dense)))
            a_block.add_line(start, end)
            count +=1
    # Forth add Arcs
    arcs = template_entities["ARC"]
    for arc in arcs:
        centre_point = arc[0]
        radius = arc[1]
        start_angle = arc[2]
        end_angle = arc[3]
        width = arc[4]
        how_wide=width[0]
        how_dense=width[1]
        count = 0
        while count <= (how_wide * how_dense)/2:
            new_radius = (radius - count * (1/how_dense))
            a_block.add_arc(centre_point, radius=new_radius, start_angle=start_angle, end_angle=end_angle)
            count +=1
        count = 0
        while count <= (how_wide * how_dense)/2:
            new_radius = (radius + count * (1/how_dense))
            a_block.add_arc(centre_point, radius=new_radius, start_angle=start_angle, end_angle=end_angle)
            count +=1
            
    msp = snag_report_drawing.modelspace()
    # Add a block reference to the block at the coordinates.
    msp.add_blockref(block_name, (210 * page_number,0), dxfattribs={
        'xscale': 1,
        'yscale': 1,
        'rotation': 0
    })
    # Add a paperspace for the block.
    new_layout = snag_report_drawing.page_setup(str(page_number), "ISO A4", landscape=False)
    new_layout.add_viewport(
        center=(210/2, 297/2),
        size=(210, 297),
        view_center_point=(210*page_number + 210/2,297/2),
        view_height=(297/1.05),
         )
    
    # save the DXF document
    snag_report_drawing.saveas(drawing_file_with_path)

template_full_page = {
    "TXT_PARAM": (2,1,6,5),
    "LINE": [[(16.5,277.91),(16.5,195.178666666667),global_line_density],
             [(74.8333333333333,277.91),(74.8333333333333,195.178666666667),global_line_density],
             [(75.8333333333333,277.91),(75.8333333333333,195.178666666667),global_line_density],
             [(134.166666666667,277.91),(134.166666666667,195.178666666667),global_line_density],
             [(135.166666666667,277.91),(135.166666666667,195.178666666667),global_line_density],
             [(193.5,277.91),(193.5,195.178666666667),global_line_density],
             [(16.5,194.178666666667),(16.5,111.447333333333),global_line_density],
             [(74.8333333333333,194.178666666667),(74.8333333333333,111.447333333333),global_line_density],
             [(75.8333333333333,194.178666666667),(75.8333333333333,111.447333333333),global_line_density],
             [(134.166666666667,194.178666666667),(134.166666666667,111.447333333333),global_line_density],
             [(135.166666666667,194.178666666667),(135.166666666667,111.447333333333),global_line_density],
             [(193.5,194.178666666667),(193.5,111.447333333333),global_line_density],
             [(16.5,110.447333333333),(16.5,27.7160000000002),global_line_density],
             [(74.8333333333333,110.447333333333),(74.8333333333333,27.7160000000002),global_line_density],
             [(75.8333333333333,110.447333333333),(75.8333333333333,27.7160000000002),global_line_density],
             [(134.166666666667,110.447333333333),(134.166666666667,27.7160000000002),global_line_density],
             [(135.166666666667,110.447333333333),(135.166666666667,27.7160000000002),global_line_density],
             [(193.5,110.447333333333),(193.5,27.7160000000002),global_line_density],
             [(16.5,277.91),(74.8333333333333,277.91),global_line_density],
             [(75.8333333333333,277.91),(134.166666666667,277.91),global_line_density],
             [(135.166666666667,277.91),(193.5,277.91),global_line_density],
             [(16.5,195.178666666667),(74.8333333333333,195.178666666667),global_line_density],
             [(75.8333333333333,195.178666666667),(134.166666666667,195.178666666667),global_line_density],
             [(135.166666666667,195.178666666667),(193.5,195.178666666667),global_line_density],
             [(16.5,194.178666666667),(74.8333333333333,194.178666666667),global_line_density],
             [(75.8333333333333,194.178666666667),(134.166666666667,194.178666666667),global_line_density],
             [(135.166666666667,194.178666666667),(193.5,194.178666666667),global_line_density],
             [(16.5,111.447333333333),(74.8333333333333,111.447333333333),global_line_density],
             [(75.8333333333333,111.447333333333),(134.166666666667,111.447333333333),global_line_density],
             [(135.166666666667,111.447333333333),(193.5,111.447333333333),global_line_density],
             [(16.5,110.447333333333),(74.8333333333333,110.447333333333),global_line_density],
             [(75.8333333333333,110.447333333333),(134.166666666667,110.447333333333),global_line_density],
             [(135.166666666667,110.447333333333),(193.5,110.447333333333),global_line_density],
             [(16.5,27.7160000000002),(74.8333333333333,27.7160000000002),global_line_density],
             [(75.8333333333333,27.7160000000002),(134.166666666667,27.7160000000002),global_line_density],
             [(135.166666666667,27.7160000000002),(193.5,27.7160000000002),global_line_density],
             [(16.5,219.57666666667),(74.8333333333333,219.57666666667),global_line_density],
             [(75.8333333333333,219.57666666667),(134.166666666667,219.57666666667),global_line_density],
             [(135.166666666667,219.57666666667),(193.5,219.57666666667),global_line_density],
             [(16.5,135.845333336667),(74.8333333333333,135.845333336667),global_line_density],
             [(75.8333333333333,135.845333336667),(134.166666666667,135.845333336667),global_line_density],
             [(135.166666666667,135.845333336667),(193.5,135.845333336667),global_line_density],
             [(16.5,52.1140000000034),(74.8333333333333,52.1140000000034),global_line_density],
             [(75.8333333333333,52.1140000000034),(134.166666666667,52.1140000000034),global_line_density],
             [(135.166666666667,52.1140000000034),(193.5,52.1140000000034),global_line_density],
             [(16.5,26.7160000000002),(193.5,26.7160000000002),global_line_density],
             [(19,13.09),(191,13.09),global_line_density],
             [(16.5,26.7160000000002),(16.5,15.59),global_line_density],
             [(193.5,26.7160000000002),(193.5,15.59),global_line_density],
             [(16.5,278.91),(193.5,278.91),global_line_density],
             [(19,283.91),(191,283.91),global_line_density],
             [(16.5,278.91),(16.5,281.41),global_line_density],
             [(193.5,278.91),(193.5,281.41),global_line_density],
             [(16.5,199.57666666667),(74.8333333333333,199.57666666667),(.05,50)],
             [(75.8333333333333,199.57666666667),(134.166666666667,199.57666666667),(.05,50)],
             [(135.166666666667,199.57666666667),(193.5,199.57666666667),(.05,50)],
             [(16.5,115.845333336667),(74.8333333333333,115.845333336667),(.05,50)],
             [(75.8333333333333,115.845333336667),(134.166666666667,115.845333336667),(.05,50)],
             [(135.166666666667,115.845333336667),(193.5,115.845333336667),(.05,50)],
             [(16.5,32.1140000000034),(74.8333333333333,32.1140000000034),(.05,50)],
             [(75.8333333333333,32.1140000000034),(134.166666666667,32.1140000000034),(.05,50)],
             [(135.166666666667,32.1140000000034),(193.5,32.1140000000034),(.05,50)]
            ],
    "ARC": [[(19,281.41),2.5,90,180,global_line_density],
            [(19,15.59),2.5,180,270,global_line_density],
            [(191,281.41),2.5,0,90,global_line_density],
            [(191,15.59),2.5,270,0,global_line_density],
           ],
    
    "HATCH": [],

    "SNAG_IMAGE": [[[(16.5,219.576),(74.833,277.91),(16.5,219.576),(16.5,277.91),(74.833,277.91),(74.833,219.576),(16.5,219.576)],'default'],
                   [[(75.833,219.576),(134.166,277.91),(75.833,219.576),(75.833,277.91),(134.166,277.91),(134.166,219.576),(75.833,219.576)],'default'],
                   [[(135.166,219.576),(193.5,277.91),(135.166,219.576),(135.166,277.91),(193.5,277.91),(193.5,219.576),(135.166,219.576)],'default'],                   
                   [[(16.5,135.845),(74.833,194.178),(16.5,135.845),(16.5,194.178),(74.8333,194.178),(74.833,135.845),(16.5,135.845)],'default'],
                   [[(75.833,135.845),(134.166,194.178),(75.833,135.845),(75.833,194.178),(134.166,194.178),(134.166,135.845),(75.833,135.845)],'default'],
                   [[(135.166,135.845),(193.5,194.178),(135.166,135.845),(135.166,194.178),(193.5,194.178),(193.5,135.845),(135.166,135.845)],'default'],
                   [[(16.5,52.114),(74.833,110.447),(16.5,52.114),(16.5,110.447),(74.833,110.447),(74.833,52.114),(16.5,52.114)],'default'],
                   [[(75.833,52.114),(134.166,110.4473),(75.833,52.114),(75.833,110.447),(134.166,110.447),(134.166,52.1141),(75.833,52.114)],'default'],
                   [[(135.166,52.114),(193.5,110.447),(135.166,52.114),(135.166,110.447),(193.5,110.447),(193.5,52.114),(135.166,52.114)],'default']
                  ],
    "SNAG_TEXT": [[(16.5,219.576),(74.833,195.178),"default",('default','default','default',('default','default')),'default'],
                  [(75.833,219.576),(134.166,195.178),"default",('default','default','default',('default','default')),'default'],
                  [(135.166,219.576),(193.499,195.178),"default",('default','default','default',('default','default')),'default'],                  
                  [(16.5,135.845),(74.833,111.447),"default",('default','default','default',('default','default')),'default'],
                  [(75.833,135.845),(134.166,111.447),"default",('default','default','default',('default','default')),'default'],
                  [(135.166,135.845),(193.499,111.447),"default",('default','default','default',('default','default')),'default'],                  
                  [(16.5,52.114),(74.833,27.7162),"default",('default','default','default',('default','default')),'default'],
                  [(75.833,52.114),(134.166,27.716),"default",('default','default','default',('default','default')),'default'],
                  [(135.166,52.114),(193.499,27.716),"default",('default','default','default',('default','default')),'default']
                 ],
    
    "SCORE_TEXT":[[(0,0),["",40,7]]
                 ],

    "WORK_SCOPE": [[(16.5+2.2,219.576-22.2)   ,(8.5,0),[["",0]]],
                   [(75.833+2.2,219.576-22.2) ,(8.5,0),[["",0]]],
                   [(135.166+2.2,219.576-22.2),(8.5,0),[["",0]]],
                   [(16.5+2.2,135.845-22.2)   ,(8.5,0),[["",0]]],
                   [(75.833+2.2,135.845-22.2) ,(8.5,0),[["",0]]],
                   [(135.166+2.2,135.845-22.2),(8.5,0),[["",0]]],
                   [(16.5+2.2,52.114-22.2)    ,(8.5,0),[["",0]]],
                   [(75.833+2.2,52.114-22.2)  ,(8.5,0),[["",0]]],
                   [(135.166+2.2,52.114-22.2) ,(8.5,0),[["",0]]],
                   
                   [(16.5+2.2,26.716-2.27),(0,0),[["",0]]],
                   [(16.5+2.2,26.716-2.27-4.54),(0,0),[["",0]]],
                   [(16.5+2.2,26.716-2.27-2*4.54),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27-4.54),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27-2*4.54),(0,0),[["",0]]],
                  ],
    "BADGE": [[(45.666,277.91)  ,(0,100),[["",0]]],
              [(104.999,277.91) ,(0,100),[["",0]]],
              [(164.333,277.91) ,(0,100),[["",0]]],
              [(45.666,194.178) ,(0,100),[["",0]]],
              [(104.999,194.178),(0,100),[["",0]]],
              [(164.333,194.178),(0,100),[["",0]]],
              [(45.666,110.447) ,(0,100),[["",0]]],
              [(104.999,110.447),(0,100),[["",0]]],
              [(164.333,110.447),(0,100),[["",0]]],
             ],
    "STICKER": [[(45.666,272.91)  ,(0,-7.5),[["",0]]],
                [(104.999,272.91) ,(0,-7.5),[["",0]]],
                [(164.333,272.91) ,(0,-7.5),[["",0]]],
                [(45.666,189.178) ,(0,-7.5),[["",0]]],
                [(104.999,189.178),(0,-7.5),[["",0]]],
                [(164.333,189.178),(0,-7.5),[["",0]]],
                [(45.666,105.447) ,(0,-7.5),[["",0]]],
                [(104.999,105.447),(0,-7.5),[["",0]]],
                [(164.333,105.447),(0,-7.5),[["",0]]],
               ],
    }

template_half_page = {
    "TXT_PARAM": (2,1,6,5),
    "LINE": [[(16.5,278.91),(16.5,195.178666666667),global_line_density],
             [(193.5,278.91),(193.5,195.178666666667),global_line_density],
             [(135.166666666667,283.91),(135.166666666667,195.178666666667),global_line_density],
             [(134.166666666667,283.91),(134.166666666667,195.178666666667),global_line_density],             
             [(135.166666666667,273.91),(193.5,273.91),global_line_density],
             #[(105,271.41),(193.5,271.41),(.05,50)],
             #[(105,268.91),(193.5,268.91),(.05,50)],
             #[(105,266.41),(193.5,266.41),(.05,50)],
             #[(105,263.91),(193.5,263.91),(.05,50)],
             [(16.5,194.178666666667),(16.5,111.447333333333),global_line_density],
             [(74.8333333333333,194.178666666667),(74.8333333333333,111.447333333333),global_line_density],
             [(75.8333333333333,194.178666666667),(75.8333333333333,111.447333333333),global_line_density],
             [(134.166666666667,194.178666666667),(134.166666666667,111.447333333333),global_line_density],
             [(135.166666666667,194.178666666667),(135.166666666667,111.447333333333),global_line_density],
             [(193.5,194.178666666667),(193.5,111.447333333333),global_line_density],
             [(16.5,110.447333333333),(16.5,27.7160000000002),global_line_density],
             [(74.8333333333333,110.447333333333),(74.8333333333333,27.7160000000002),global_line_density],
             [(75.8333333333333,110.447333333333),(75.8333333333333,27.7160000000002),global_line_density],
             [(134.166666666667,110.447333333333),(134.166666666667,27.7160000000002),global_line_density],
             [(135.166666666667,110.447333333333),(135.166666666667,27.7160000000002),global_line_density],
             [(193.5,110.447333333333),(193.5,27.7160000000002),global_line_density],

             [(16.5,195.178666666667),(134.166666666667,195.178666666667),global_line_density],
             [(135.166666666667,195.178666666667),(193.5,195.178666666667),global_line_density],
             
             [(16.5,194.178666666667),(74.8333333333333,194.178666666667),global_line_density],
             [(75.8333333333333,194.178666666667),(134.166666666667,194.178666666667),global_line_density],
             [(135.166666666667,194.178666666667),(193.5,194.178666666667),global_line_density],
             [(16.5,111.447333333333),(74.8333333333333,111.447333333333),global_line_density],
             [(75.8333333333333,111.447333333333),(134.166666666667,111.447333333333),global_line_density],
             [(135.166666666667,111.447333333333),(193.5,111.447333333333),global_line_density],
             [(16.5,110.447333333333),(74.8333333333333,110.447333333333),global_line_density],
             [(75.8333333333333,110.447333333333),(134.166666666667,110.447333333333),global_line_density],
             [(135.166666666667,110.447333333333),(193.5,110.447333333333),global_line_density],
             [(16.5,27.7160000000002),(74.8333333333333,27.7160000000002),global_line_density],
             [(75.8333333333333,27.7160000000002),(134.166666666667,27.7160000000002),global_line_density],
             [(135.166666666667,27.7160000000002),(193.5,27.7160000000002),global_line_density],
             #[(16.5,219.57666666667),(74.8333333333333,219.57666666667),global_line_density],
             #[(75.8333333333333,219.57666666667),(134.166666666667,219.57666666667),global_line_density],
             #[(135.166666666667,219.57666666667),(193.5,219.57666666667),global_line_density],
             [(16.5,135.845333336667),(74.8333333333333,135.845333336667),global_line_density],
             [(75.8333333333333,135.845333336667),(134.166666666667,135.845333336667),global_line_density],
             [(135.166666666667,135.845333336667),(193.5,135.845333336667),global_line_density],
             [(16.5,52.1140000000034),(74.8333333333333,52.1140000000034),global_line_density],
             [(75.8333333333333,52.1140000000034),(134.166666666667,52.1140000000034),global_line_density],
             [(135.166666666667,52.1140000000034),(193.5,52.1140000000034),global_line_density],
             [(16.5,26.7160000000002),(193.5,26.7160000000002),global_line_density],
             [(19,13.09),(191,13.09),global_line_density],
             [(16.5,26.7160000000002),(16.5,15.59),global_line_density],
             [(193.5,26.7160000000002),(193.5,15.59),global_line_density],
             #[(105,278.91),(193.5,278.91),global_line_density],

             [(19,283.91),(134.166666666667,283.91),global_line_density],
             [(135.166666666667,283.91),(191,283.91),global_line_density],             

             [(135.166666666667,239.5443333333335),(193.5,239.5443333333335),global_line_density],
             
             [(16.5,278.91),(16.5,281.41),global_line_density],
             [(193.5,278.91),(193.5,281.41),global_line_density],
             #[(16.5,199.57666666667),(74.8333333333333,199.57666666667),(.05,50)],
             #[(75.8333333333333,199.57666666667),(134.166666666667,199.57666666667),(.05,50)],
             #[(135.166666666667,199.57666666667),(193.5,199.57666666667),(.05,50)],
             [(16.5,115.845333336667),(74.8333333333333,115.845333336667),(.05,50)],
             [(75.8333333333333,115.845333336667),(134.166666666667,115.845333336667),(.05,50)],
             [(135.166666666667,115.845333336667),(193.5,115.845333336667),(.05,50)],
             [(16.5,32.1140000000034),(74.8333333333333,32.1140000000034),(.05,50)],
             [(75.8333333333333,32.1140000000034),(134.166666666667,32.1140000000034),(.05,50)],
             [(135.166666666667,32.1140000000034),(193.5,32.1140000000034),(.05,50)]
            ],
    "ARC": [[(19,281.41),2.5,90,180,global_line_density],
            [(19,15.59),2.5,180,270,global_line_density],
            [(191,281.41),2.5,0,90,global_line_density],
            [(191,15.59),2.5,270,0,global_line_density],
           ],
    
    "HATCH": [[(17.5,196.178666666667),(74.83333333,196.178666666667),(74.83333333,239.044333333),(17.5,239.044333333)],
              [(75.83333333,196.178666666667),(133.166666666667,196.178666666667),(133.166666666667,239.044333333),(75.83333333,239.044333333)],
              [(17.5,240.044333333),(74.83333333,240.044333333),(74.83333333,282.91),(17.5,282.91)],
              [(75.83333333,240.044333333),(133.166666666667,240.044333333),(133.166666666667,282.91),(75.83333333,282.91)],
             ],    
  
    "SNAG_IMAGE": [[[(135.166666666667,195.178666666667),(193.5,239.5443333333335),(135.166666666667,195.178666666667),(135.166666666667,239.5443333333335),(193.5,239.5443333333335),(193.5,195.178666666667),(135.166666666667,195.178666666667)],'default'],

                   [[(75.83333333,240.044333333),(133.166666666667,282.91),(75.83333333,240.044333333),(133.166666666667,240.044333333),(133.166666666667,282.91),(75.83333333,282.91)],'default'],
                   [[(17.5,240.044333333),(74.83333333,282.91),(17.5,240.044333333),(74.83333333,240.044333333),(74.83333333,282.91),(17.5,282.91)],'default'],
                   [[(75.83333333,196.178666666667),(133.166666666667,239.044333333),(75.83333333,196.178666666667),(133.166666666667,196.178666666667),(133.166666666667,239.044333333),(75.83333333,239.044333333)],'default'],
                   [[(17.5,196.178666666667),(74.83333333,239.044333333),(17.5,196.178666666667),(74.83333333,196.178666666667),(74.83333333,239.044333333),(17.5,239.044333333)],'default'],                   
                   
                   [[(16.5,135.845333333333),(74.8333333333333,194.178666666667),(16.5,135.845333333333),(16.5,194.178666666667),(74.8333333333333,194.178666666667),(74.8333333333333,135.845333333333),(16.5,135.845333333333)],'default'],
                   [[(75.8333333333333,135.845333333333),(134.166666666667,194.178666666667),(75.8333333333333,135.845333333333),(75.8333333333333,194.178666666667),(134.166666666667,194.178666666667),(134.166666666667,135.845333333333),(75.8333333333333,135.845333333333)],'default'],
                   [[(135.166666666667,135.845333333333),(193.5,194.178666666667),(135.166666666667,135.845333333333),(135.166666666667,194.178666666667),(193.5,194.178666666667),(193.5,135.845333333333),(135.166666666667,135.845333333333)],'default'],                   
                   [[(16.5,52.1140000000001),(74.8333333333333,110.447333333333),(16.5,52.1140000000001),(16.5,110.447333333333),(74.8333333333333,110.447333333333),(74.8333333333333,52.1140000000001),(16.5,52.1140000000001)],'default'],
                   [[(75.8333333333333,52.1140000000001),(134.166666666667,110.447333333333),(75.8333333333333,52.1140000000001),(75.8333333333333,110.447333333333),(134.166666666667,110.447333333333),(134.166666666667,52.1140000000001),(75.8333333333333,52.1140000000001)],'default'],
                   [[(135.166666666667,52.1140000000001),(193.5,110.447333333333),(135.166666666667,52.1140000000001),(135.166666666667,110.447333333333),(193.5,110.447333333333),(193.5,52.1140000000001),(135.166666666667,52.1140000000001)],'default']
                  ],
    "SNAG_TEXT": [[(163.25,280.41),(163.25,280.41),"default",('default','default','default',('default','default')),'default'],

                  [(163.25,280.41),(163.25,280.41),"default",('default','default','default',('default','default')),'default'],
                  [(163.25,280.41),(163.25,280.41),"default",('default','default','default',('default','default')),'default'],
                  [(163.25,280.41),(163.25,280.41),"default",('default','default','default',('default','default')),'default'],
                  [(163.25,280.41),(163.25,280.41),"default",('default','default','default',('default','default')),'default'],                  
                  
                  [(16.5,135.845333366667),(74.8333333333333,111.447333333333),"default",('default','default','default',('default','default')),'default'],
                  [(75.833333333333,135.845333366667),(134.166666666666,111.447333333333),"default",('default','default','default',('default','default')),'default'],
                  [(135.166666666666,135.845333366667),(193.499999999999,111.447333333333),"default",('default','default','default',('default','default')),'default'],
                  [(16.5,52.1140000333335),(74.8333333333333,27.7160000000002),"default",('default','default','default',('default','default')),'default'],
                  [(75.833333333333,52.1140000333335),(134.166666666666,27.7160000000002),"default",('default','default','default',('default','default')),'default'],
                  [(135.166666666666,52.1140000333335),(193.499999999999,27.7160000000002),"default",('default','default','default',('default','default')),'default']
                 ],

    "SCORE_TEXT":[[(0,0),["",40,7]]
                 ],    
    
     
    "WORK_SCOPE": [[(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(16.5+2.2,135.845-22.2)   ,(8.5,0),[["",0]]],
                   [(75.833+2.2,135.845-22.2) ,(8.5,0),[["",0]]],
                   [(135.166+2.2,135.845-22.2),(8.5,0),[["",0]]],
                   [(16.5+2.2,52.114-22.2)    ,(8.5,0),[["",0]]],
                   [(75.833+2.2,52.114-22.2)  ,(8.5,0),[["",0]]],
                   [(135.166+2.2,52.114-22.2) ,(8.5,0),[["",0]]],
                   
                   [(16.5+2.2,26.716-2.27),(0,0),[["",0]]],
                   [(16.5+2.2,26.716-2.27-4.54),(0,0),[["",0]]],
                   [(16.5+2.2,26.716-2.27-2*4.54),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27-4.54),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27-2*4.54),(0,0),[["",0]]],
                  ],

    "BADGE": [[(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(45.666,194.178) ,(0,100),[["",0]]],
              [(104.999,194.178),(0,100),[["",0]]],
              [(164.333,194.178),(0,100),[["",0]]],
              [(45.666,110.447) ,(0,100),[["",0]]],
              [(104.999,110.447),(0,100),[["",0]]],
              [(164.333,110.447),(0,100),[["",0]]],
             ],
    "STICKER": [[(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(45.666,189.178) ,(0,-7.5),[["",0]]],
                [(104.999,189.178),(0,-7.5),[["",0]]],
                [(164.333,189.178),(0,-7.5),[["",0]]],
                [(45.666,105.447) ,(0,-7.5),[["",0]]],
                [(104.999,105.447),(0,-7.5),[["",0]]],
                [(164.333,105.447),(0,-7.5),[["",0]]],
               ],
}
XX=89.5 + 5
YY=220 + 5 #180
score_x = 28.7685
score_y = 215.8342
template_cover_page = {
    "TXT_PARAM": (4,6),
    "LINE": [[(16.5,281.41),(16.5,15.59),global_line_density],
             [(193.5,281.41),(193.5,15.59),global_line_density],
             [(19,283.91),(191,283.91),global_line_density],
             [(19,13.09),(191,13.09),global_line_density],
             [(89.5,175.0),(187.5,175.0),global_line_density],
             [(40,220),(40,105),global_line_density],
            ],
    "ARC": [[(19,281.41),2.5,90,180,global_line_density],
            [(19,15.59),2.5,180,270,global_line_density],
            [(191,281.41),2.5,0,90,global_line_density],
            [(191,15.59),2.5,270,0,global_line_density],
           ],
    "HATCH": [[(17.5,YY),(87.5+5,YY),(87.5+5,YY+40+18),(17.5,YY+40+18)],

             ],
    "SNAG_IMAGE": [[[(XX,YY),(XX+98,YY+58),(XX+(18+2)*0,YY+(18+2)*0),(XX+(18+2)*0,YY+(18+2)*0+18),(XX+(18+2)*0+18,YY+(18+2)*0+18),(XX+(18+2)*0+18,YY+(18+2)*0),(XX+(18+2)*0,YY+(18+2)*0)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*1,YY+(18+2)*0),(XX+(18+2)*1,YY+(18+2)*0+18),(XX+(18+2)*1+18,YY+(18+2)*0+18),(XX+(18+2)*1+18,YY+(18+2)*0),(XX+(18+2)*1,YY+(18+2)*0)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*2,YY+(18+2)*0),(XX+(18+2)*2,YY+(18+2)*0+18),(XX+(18+2)*2+18,YY+(18+2)*0+18),(XX+(18+2)*2+18,YY+(18+2)*0),(XX+(18+2)*2,YY+(18+2)*0)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*3,YY+(18+2)*0),(XX+(18+2)*3,YY+(18+2)*0+18),(XX+(18+2)*3+18,YY+(18+2)*0+18),(XX+(18+2)*3+18,YY+(18+2)*0),(XX+(18+2)*3,YY+(18+2)*0)],'default'],                   
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*4,YY+(18+2)*0),(XX+(18+2)*4,YY+(18+2)*0+18),(XX+(18+2)*4+18,YY+(18+2)*0+18),(XX+(18+2)*4+18,YY+(18+2)*0),(XX+(18+2)*4,YY+(18+2)*0)],'default'],
                   
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*0,YY+(18+2)*1),(XX+(18+2)*0,YY+(18+2)*1+18),(XX+(18+2)*0+18,YY+(18+2)*1+18),(XX+(18+2)*0+18,YY+(18+2)*1),(XX+(18+2)*0,YY+(18+2)*1)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*1,YY+(18+2)*1),(XX+(18+2)*1,YY+(18+2)*1+18),(XX+(18+2)*1+18,YY+(18+2)*1+18),(XX+(18+2)*1+18,YY+(18+2)*1),(XX+(18+2)*1,YY+(18+2)*1)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*2,YY+(18+2)*1),(XX+(18+2)*2,YY+(18+2)*1+18),(XX+(18+2)*2+18,YY+(18+2)*1+18),(XX+(18+2)*2+18,YY+(18+2)*1),(XX+(18+2)*2,YY+(18+2)*1)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*3,YY+(18+2)*1),(XX+(18+2)*3,YY+(18+2)*1+18),(XX+(18+2)*3+18,YY+(18+2)*1+18),(XX+(18+2)*3+18,YY+(18+2)*1),(XX+(18+2)*3,YY+(18+2)*1)],'default'],                   
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*4,YY+(18+2)*1),(XX+(18+2)*4,YY+(18+2)*1+18),(XX+(18+2)*4+18,YY+(18+2)*1+18),(XX+(18+2)*4+18,YY+(18+2)*1),(XX+(18+2)*4,YY+(18+2)*1)],'default'],            

                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*0,YY+(18+2)*2),(XX+(18+2)*0,YY+(18+2)*2+18),(XX+(18+2)*0+18,YY+(18+2)*2+18),(XX+(18+2)*0+18,YY+(18+2)*2),(XX+(18+2)*0,YY+(18+2)*2)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*1,YY+(18+2)*2),(XX+(18+2)*1,YY+(18+2)*2+18),(XX+(18+2)*1+18,YY+(18+2)*2+18),(XX+(18+2)*1+18,YY+(18+2)*2),(XX+(18+2)*1,YY+(18+2)*2)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*2,YY+(18+2)*2),(XX+(18+2)*2,YY+(18+2)*2+18),(XX+(18+2)*2+18,YY+(18+2)*2+18),(XX+(18+2)*2+18,YY+(18+2)*2),(XX+(18+2)*2,YY+(18+2)*2)],'default'],
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*3,YY+(18+2)*2),(XX+(18+2)*3,YY+(18+2)*2+18),(XX+(18+2)*3+18,YY+(18+2)*2+18),(XX+(18+2)*3+18,YY+(18+2)*2),(XX+(18+2)*3,YY+(18+2)*2)],'default'],                   
                   [[(XX,YY),(XX+98,YY+58),(XX+(18+2)*4,YY+(18+2)*2),(XX+(18+2)*4,YY+(18+2)*2+18),(XX+(18+2)*4+18,YY+(18+2)*2+18),(XX+(18+2)*4+18,YY+(18+2)*2),(XX+(18+2)*4,YY+(18+2)*2)],'default'],
                   [[(187.5-38+5,19.09-5),(187.5+5,19.09+38-5),(187.5-38+5,19.09-5),(187.5-38+5,19.09+38-5),(187.5+5,19.09+38-5),(187.5+5,19.09-5),(187.5-38+5,19.09-5)],'default'],                   
                  ],
    "SNAG_TEXT": [[(186.5,180),(186.5,180),"default",('default','default','default',('default','default')),'default'],
                  [(186.5,172.5),(186.5,172.5),"default",('default','default','default',('default','default')),'default'],
                  [(186.5,165),(186.5,165),"default",('default','default','default',('default','default')),'default'],
                  [(186.5,157.5),(186.5,157.5),"default",('default','default','default',('default','default')),'default'],
                  [(186.5,150),(186.5,150),"default",('default','default','default',('default','default')),'default'],
                  [(186.5,19.09+38+12.5),(186.5,19.09+38+12.5),"default",('default','default','default',('default','default')),'default'],
                  [(186.5,19.09+38+5),(186.5,19.09+38+5),"default",('default','default','default',('default','default')),'default'],
                 ],

    "SCORE_TEXT":[[(55,254),["01",40,7]],
                  [(score_x, score_y-15*0),["00",10,254]],
                  [(score_x, score_y-15*1),["00",10,254]],
                  [(score_x, score_y-15*2),["00",10,254]],
                  [(score_x, score_y-15*3),["00",10,254]],
                  [(score_x, score_y-15*4),["00",10,254]],
                  [(score_x, score_y-15*5),["00",10,254]],
                  [(score_x, score_y-15*6),["00",10,254]],
                  [(score_x, score_y-15*7),["00",10,254]],
                 ],


    "WORK_SCOPE": [[(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                   [(0,0)            ,(0,0),[["",0]]],
                      
                   [(16.5+2.2,26.716-2.27),(0,0),[["",0]]],
                   [(16.5+2.2,26.716-2.27-4.54),(0,0),[["",0]]],
                   [(16.5+2.2,26.716-2.27-2*4.54),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27-4.54),(0,0),[["",0]]],
                   [(75.833+2.2,26.716-2.27-2*4.54),(0,0),[["",0]]],                  
                  ],
    "BADGE": [[(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
              [(0,0)            ,(0,0),[["",0]]],
             ],
    
    "STICKER": [[(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
                [(0,0)            ,(0,0),[["",0]]],
               ],

}
    
work_scope_template = [[["RED", "red"],"ELECTRICAL: LIGHTING, POWER, WIREWAYS",1],
                       [["YELLOW", "yellow"],"ELECTRONIC: DATA, ACCESS CONTROL, PA, ALARM",2],
                       [["GREEN", "green"],"BUILDER",3],
                       [["CYAN", "cyan"],"BCX",4],
                       [["BLUE", "blue"],"N/A",5],
                       [["MAGENTA", "magenta"],"N/A",6],
                       #[["WHITE", "white"],"",7],
                       #[["BLACK", "black"],"ELECTRICAL ENGINEER, FACILITIES MANAGEMENT, PROPERTY SERVICES, QS",7],
                       #[["GREY", "grey"],"",9]
                      ]

badge_template = [[[0,25,50,75,100],"IN PROGRESS",1]
                 ]

sticker_template = [[["DONE"],"DONE",2],
                    [["CANCELLED"],"CANCELLED",2],
                    [["ENG"],"ENG-CI",2]
                   ]

In [9]:
#def create_snag_summary_template
#def create_snag_photo_template(template_type)

root_directory="C:\\Users\\Carlos de Gouveia\\My Drive\\Projects\\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\\Snagging"
list_of_snag_images = find_snagged_image_files(root_directory)
pages=[]
current_page = copy.deepcopy(template_cover_page)
current_page["SNAG_TEXT"][0][2] = "INSPECTION LIST REVISION #1"
current_page["SNAG_TEXT"][1][2] = "SWELLENDAM RAILTON CDC"
current_page["SNAG_TEXT"][2][2] = "GENERAL MAINTENANCE (ALPHA)"
current_page["SNAG_TEXT"][3][2] = "ELECTRICAL AND ELECTRONIC SERVICES"
current_page["SNAG_TEXT"][4][2] = "MS/810001-0109"
current_page["SNAG_TEXT"][5][2] = "E5177"
current_page["SNAG_TEXT"][6][2] = datetime.date.today().strftime("%d %B %Y")
count = 0
for row in current_page["SNAG_IMAGE"]:
    current_page["SNAG_IMAGE"][count][1] = "C:\\Users\\Carlos de Gouveia\\My Drive\\Projects\\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Photos\\2023-07-24\\1690195276.214.JPG"
    count +=1
count -=1
current_page["SNAG_IMAGE"][count][1] = "C:\\Users\\Carlos de Gouveia\\My Drive\\Logo\\dvm lodo DVM- Monochrome logo.jpg"

pages.append(copy.deepcopy(current_page))        
#current_page = copy.deepcopy(template_room_page)
#max_image_count = len(current_page["SNAG_IMAGE"])
image_number = 0
location_name = ""
project_score = create_work_scope_score(work_scope_template, sticker_template)
for image in list_of_snag_images:
    print(root_directory)
    if get_location_tree_from_path(root_directory, image)[0] != location_name:
        if location_name != "":
            pages.append(copy.deepcopy(current_page))
        current_page = copy.deepcopy(template_half_page)
        title_image = root_directory + "\\" + get_location_tree_from_path(root_directory, image)[0] + "\\[TITLE].JPEG"
        title_image_panel1 = root_directory + "\\" + get_location_tree_from_path(root_directory, image)[0] + "\\[TITLE_PANEL1].JPG"
        title_image_panel2 = root_directory + "\\" + get_location_tree_from_path(root_directory, image)[0] + "\\[TITLE_PANEL2].JPG"        
        title_image_panel3 = root_directory + "\\" + get_location_tree_from_path(root_directory, image)[0] + "\\[TITLE_PANEL3].JPG"        
        title_image_panel4 = root_directory + "\\" + get_location_tree_from_path(root_directory, image)[0] + "\\[TITLE_PANEL4].JPG"
        location_name = get_location_tree_from_path(root_directory, image)[0]
        room_score = create_work_scope_score(work_scope_template, sticker_template)
        for score_image in list_of_snag_images:
            if get_location_tree_from_path(root_directory, score_image)[0] == location_name:
                keywords = get_snag_keyword_from_title(score_image)
                room_score = keep_score_work_scope(work_scope_template, sticker_template, keywords, room_score)
        count = 0
        for a_list in project_score:
            project_score[count] = [project_score[count][i] + room_score[count][i] for i in range(len(project_score[count]))]
            count += 1
        print(room_score)
        print(project_score)
        
        current_page["SNAG_IMAGE"][0][1] = title_image
        current_page["SNAG_TEXT"][0][2] = location_name  
        current_page["SNAG_TEXT"][0][4] = "TITLE"

        current_page["SNAG_IMAGE"][1][1] = title_image_panel1
        current_page["SNAG_TEXT"][1][2] = location_name  
        current_page["SNAG_TEXT"][1][4] = "TITLE"

        current_page["SNAG_IMAGE"][2][1] = title_image_panel2
        current_page["SNAG_TEXT"][2][2] = location_name  
        current_page["SNAG_TEXT"][2][4] = "TITLE"
        
        current_page["SNAG_IMAGE"][3][1] = title_image_panel3
        current_page["SNAG_TEXT"][3][2] = location_name  
        current_page["SNAG_TEXT"][3][4] = "TITLE"        

        current_page["SNAG_IMAGE"][4][1] = title_image_panel4
        current_page["SNAG_TEXT"][4][2] = location_name  
        current_page["SNAG_TEXT"][4][4] = "TITLE"        
        
        max_image_count = len(current_page["SNAG_IMAGE"])
        image_number = 5
    elif image_number == max_image_count:
        key_count = 0
        for work_scope_key in work_scope_template:
            #print(work_scope_key)
            current_page["WORK_SCOPE"][image_number+key_count][2] = [[work_scope_key[1],work_scope_key[2]]]
            key_count += 1
        pages.append(copy.deepcopy(current_page))
        current_page = copy.deepcopy(template_full_page)
        max_image_count = len(current_page["SNAG_IMAGE"])
        image_number = 0 
    current_page["SNAG_IMAGE"][image_number][1] = image
    current_page["SNAG_TEXT"][image_number][2] = get_snag_comment_from_title(image)    
    current_page["SNAG_TEXT"][image_number][3] = get_image_exif(image) 
    keywords = get_snag_keyword_from_title(image)
    current_page["WORK_SCOPE"][image_number][2] = get_work_scope(work_scope_template, keywords)

    current_page["BADGE"][image_number][2] = get_badge(badge_template,keywords)
    current_page["STICKER"][image_number][2] = get_sticker(sticker_template, keywords)    
    image_number +=1
        
pages.append(copy.deepcopy(current_page))
start_new_drawing("C:\\Users\\Carlos de Gouveia\\My Drive\\Projects\\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\\Snagging\\mytest.dxf")
page_count = 0
for current_page in pages:
    create_drawing_template("C:\\Users\\Carlos de Gouveia\\My Drive\\Projects\\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\\Snagging\\mytest.dxf", page_count, current_page)
    page_count +=1

C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Snagging
[[0, 0, 0, 0], [6, 0, 0, 0], [2, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[0, 0, 0, 0], [6, 0, 0, 0], [2, 0, 0, 0], [1, 0, 0, 0], [2, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Snagging
C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Snagging
C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Snagging
C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Snagging
C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Snagging
C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Snagging
C:\Users\Carlos de Gouveia\My Drive\Projects\E5177 - Swellendam, Railton Clinic, DTPW (CdG)\Sna

In [19]:
class MyClass:
  x = 5
p1 = MyClass()
c=5
def a(b,d):
    p1.x = b
    global c
    c=10
a(5,1)
print(p1.x, c)

5 10


In [40]:
print(work_scope_score)
work_scope_score[0][1] += 1
print(work_scope_score)

[[16, 17, 0, 6], [16, 17, 0, 6], [16, 17, 0, 6], [16, 17, 0, 6], [16, 17, 0, 6], [16, 17, 0, 6], [16, 17, 0, 6]]
[[16, 18, 0, 6], [16, 18, 0, 6], [16, 18, 0, 6], [16, 18, 0, 6], [16, 18, 0, 6], [16, 18, 0, 6], [16, 18, 0, 6]]


In [52]:
print(project_score)
print(room_score[0][0]+project_score[0][0])

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
0
